In [8]:
import pandas as pd
import time
import numpy as np
np.seterr(invalid='ignore')
import nbimporter
from PPP import PPP

prevGames = ['Game1', 'Game2', 'Game3', 'Game4']
Opp = 'Nuggets'
Team = 'Lakers'

In [4]:
players = ['davis', 'lebron', 'christie', 'reaves', 'rui', 'russell', 'schroder', 'thompson', 'vando', 'walker', 'team', 'opp']

In [6]:
def load_data(players):
    
    # Establish dictionaries for new game data and overall series/season data
    new_player_data = {}
    ovr_player_data = {}
    
    for game in prevGames: # Loop over all games in series/season
        
        for player in players: # Loop over all players on roster
            
            if player == 'team': # Add all player data together to create team file for game

                try: # to load a team file for the game (helpful if already added team file, and/or rerunning for Opp file)
                    newplayerFile = (f'Team/{game + Opp}Team.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the existing new team file to the new_player_data dictionary
                except(FileNotFoundError):
                    # Load an empty df
                    newplayerFile = ('Team/empty.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.set_index('Shot Type')

                    for df in new_player_data.values(): # Loop over each players df and add to team df
                        newplayer = newplayer.add(df)

                    new_player_data['team'] = newplayer # Add the created new team file to new_player_data dictionary
                    newplayer.to_csv(f'../{Opp}/Opp/{game + Team}Opp.csv') # Save team file to opponent folder for their opponent stats
                    newplayer.to_csv(f'Team/{game}Team.csv') # Save team file to team folder
                
                # For game 1, set the new team file as the overall and continue to the opponent
                if game[0:5] == 'Game1':
                    ovr_player_data['team'] = newplayer
                    newplayer.to_csv(f'!Data/Team.csv') # Write the new team file to the overall data folder
                    continue
                    
                # Load overall data for the team
                ovrplayerFile = (f'!Data/Team.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add the overall team data to the ovr_player_data dictionary
                    
                if newplayer.columns.equals(ovrplayer.columns): # Make sure the new and overall data frames can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer # Reassign the ovr_data_player['team'] data to the updated data
                    ovrplayer.to_csv('!Data/Team.csv') # Write the new team file to the overall data folder
                else:
                    print(f"{player.title()}'s columns do not match")
                continue # Team file is handled, so continue to the opponent file

            if player == 'opp': # Load the opponent file and add the new game data to the overall opponent file if necessary

                try: # to load the new opponent file (if running this teams data first, there will be no opponent file!!)
                    newplayerFile = (f'Opp/{game + Opp}Opp.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the new opponent file to the new_player_data dictionary
                except(FileNotFoundError):
                    print(f'No {player.title()} file found.')
                
                # For game 1, set the new opponent file as the overall and continue to the next player
                if game[0:5] == 'Game1':
                    ovr_player_data['opp'] = newplayer
                    newplayer.to_csv(f'!Data/Opp.csv') # Write the new opponent file into the overall data directory
                    continue
                    
                # Load overall data for the opponent
                ovrplayerFile = (f'!Data/Opp.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add the overall opponent file to the ovr_player_data dictionary

                if newplayer.columns.equals(ovrplayer.columns): # Make sure the new and overall dataframes can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv('!Data/Opp.csv') # Write the updated opponent file to the overall data directory
                else:
                    print(f"{player.title()}'s columns do not match")
                continue # Opponent file is handled, continue and exit the loop
                
            ### Loading/Adding the player files (first part of the loop)###
            try:
                newplayerFile = (f'{player.title()}/{game + Opp + player.title()}.csv') # Search the team directory for the player, then concat the game/opp/player to find csv for game
                newplayer = pd.read_csv(newplayerFile)
                newplayer = newplayer.dropna()
                newplayer = newplayer.set_index('Shot Type')
                newplayer = newplayer.astype('int')
                new_player_data[player] = newplayer # Add the new player data to the new_player_data dictionary
            except(FileNotFoundError):
                print(f'No {player.title()} file found for {game}.')
                try: # to load an overall file for the player
                    ovrplayerFile = (f'!Data/{player.title()}.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
                    continue
                except(FileNotFoundError):
                    print(f'No data for {player.title()}.')
                    continue # If no player data is found, skip and go to the next player (edit for later)
            
            # For game 1, set the new player file as the overall and continue to the next player
            if game == 'Game1':
                ovr_player_data[player] = newplayer
                newplayer.to_csv(f'!Data/{player.title()}.csv')
                continue
                
            # Load overall data for the player
            try: 
                ovrplayerFile = (f'!Data/{player.title()}.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
            # If there is no overall data for the player, set the new player data as the overall and continue to the next player    
            except(FileNotFoundError):
                ovrplayer = newplayer
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv')
                continue
            
            if newplayer.columns.equals(ovrplayer.columns): # Make sure the new and overall dataframes can be added without error
                ovrplayer = ovrplayer.add(newplayer)
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
            else:
                print(f"{player.title()}'s columns do not match")  
            continue
            
    return new_player_data, ovr_player_data # Return both dictionaries as a tuple

new_player_data, ovr_player_data  = load_data(players) # Unpack the tuple of dictionaries into their respective names

No Christie file found for Game1.
No data for Christie.
No Thompson file found for Game1.
No Christie file found for Game2.
No data for Christie.
No Thompson file found for Game2.
No Christie file found for Game3.
No data for Christie.
No Thompson file found for Game3.
No Christie file found for Game4.
No data for Christie.
No Vando file found for Game4.


In [11]:
newLebronStats = PPP(new_player_data['lebron'])
newLebronStats.to_csv(f'Lebron/PPP Stats/{prevGames[-1] + Opp}.csv')
newLebronStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.45,38.2,0,64.3,1.69,34.2,75.0,6,83.3,2,50.0,1.21,43.2,50.0,5,60.0,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.01,35.9,0,43.8,0.94,28.6,42.9,7,42.9,0,N/A,1.06,45.0,44.4,4,75.0,5,20.0
TRANSITION,1.81,5.9,0,100.0,1.7,6.8,N/A,0,N/A,0,N/A,2.0,4.8,100.0,1,100.0,0,N/A
Attacking Closeouts,1.0,4.3,0,50.0,2.0,3.8,100.0,1,100.0,0,N/A,0.0,4.8,0.0,0,N/A,1,0.0
Catch & Shoot,1.8,10.6,0,60.0,1.8,19.2,60.0,0,N/A,5,60.0,N/A,0.0,N/A,0,N/A,0,N/A


In [12]:
lebronStats = PPP(ovr_player_data['lebron'])
lebronStats.to_csv('!PPP Stats/lebronStats.csv')
lebronStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.28,29.5,2,54.8,1.2,22.9,55.0,14,71.4,6,16.7,1.33,37.1,54.5,16,43.8,6,83.3
PNR Screener,2.0,0.6,0,100.0,2.0,1.1,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.0,1.2,0,50.0,2.0,1.1,100.0,1,100.0,0,N/A,0.0,1.3,0.0,1,0.0,0,N/A
DHO Screener,1.04,1.7,0,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.04,3.6,50.0,2,50.0,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.13,32.8,1,46.9,1.06,31.1,45.5,17,52.9,5,20.0,1.2,34.7,48.1,11,63.6,16,37.5
TRANSITION,1.11,18.5,3,47.6,1.12,22.6,46.2,10,60.0,3,0.0,1.1,13.8,50.0,2,100.0,6,33.3
Attacking Closeouts,0.67,3.5,0,33.3,0.67,3.3,33.3,3,33.3,0,N/A,0.67,3.8,33.3,2,50.0,1,0.0
Catch & Shoot,1.25,7.1,0,41.7,1.25,13.2,41.7,0,N/A,12,41.7,N/A,0.0,N/A,0,N/A,0,N/A


In [73]:
newDavisStats = PPP(new_player_data['davis'])
newDavisStats.to_csv(f'Davis/PPP Stats/{newGame}.csv')
newDavisStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.82,40.6,1,0.0,0.82,42.6,0.0,4,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.0,4.8,0,0.0,0.0,5.0,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.4,23.8,1,75.0,1.0,20.0,66.7,3,66.7,0,N/A,3.0,100.0,100.0,0,N/A,1,100.0
TRANSITION,1.06,9.0,0,0.0,0.0,5.0,0.0,1,0.0,0,N/A,2.27,88.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,4.8,0,0.0,0.0,5.0,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [74]:
davisStats = PPP(ovr_player_data['davis'])
davisStats.to_csv('!PPP Stats/davisStats.csv')
davisStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.06,1.8,0,0.0,1.06,2.1,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.13,26.4,1,42.1,1.1,29.4,38.9,18,38.9,0,N/A,2.0,7.1,100.0,1,100.0,0,N/A
DHO Ball Handler,0.0,1.0,0,0.0,0.0,1.1,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.57,6.7,1,33.3,0.0,3.3,0.0,2,0.0,0,N/A,1.0,28.6,50.0,4,50.0,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.86,29.0,5,43.5,0.82,25.8,38.9,18,38.9,0,N/A,1.0,50.0,60.0,3,66.7,2,50.0
TRANSITION,1.72,8.9,0,50.0,1.6,8.4,50.0,4,50.0,0,N/A,2.27,12.6,N/A,0,N/A,0,N/A
Attacking Closeouts,1.29,3.7,0,66.7,1.29,4.3,66.7,3,66.7,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.5,3.8,0,50.0,1.5,4.4,50.0,0,N/A,4,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [75]:
newReavesStats = PPP(new_player_data['reaves'])
newReavesStats.to_csv(f'Reaves/PPP Stats/{newGame}.csv')
newReavesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.5,50.0,1,28.6,0.4,38.5,25.0,4,25.0,0,N/A,0.67,100.0,33.3,2,50.0,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,2.27,5.5,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,2.27,29.3,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.0,12.5,0,50.0,1.0,15.4,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.13,11.8,0,100.0,2.13,14.5,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,2.05,15.2,0,100.0,2.05,18.8,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,3.0,6.2,0,100.0,3.0,7.7,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A


In [76]:
reavesStats = PPP(ovr_player_data['reaves'])
reavesStats.to_csv('!PPP Stats/reavesStats.csv')
reavesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.97,31.2,3,47.8,0.94,25.0,36.4,8,37.5,3,33.3,1.0,41.2,58.3,8,87.5,4,0.0
PNR Screener,2.0,3.4,0,66.7,3.0,3.6,100.0,0,N/A,2,100.0,0.0,2.9,0.0,0,N/A,1,0.0
DHO Ball Handler,2.0,1.1,0,100.0,2.0,1.8,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,2.27,1.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,2.27,2.6,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.33,10.1,1,62.5,1.6,9.1,60.0,3,33.3,2,100.0,1.0,11.8,66.7,2,100.0,1,0.0
TRANSITION,1.57,20.0,1,75.0,2.26,12.1,100.0,3,100.0,1,100.0,1.16,32.9,62.5,6,66.7,2,50.0
Attacking Closeouts,1.36,11.6,0,55.6,1.29,16.9,50.0,7,42.9,1,100.0,2.0,2.9,100.0,1,100.0,0,N/A
Catch & Shoot,1.5,15.7,0,50.0,1.5,25.5,50.0,0,N/A,14,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [77]:
newRuiStats = PPP(new_player_data['rui'])
newRuiStats.to_csv(f'Rui/PPP Stats/{newGame}.csv')
newRuiStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,13.3,0,0.0,0.0,8.3,0.0,1,0.0,0,N/A,0.0,33.3,0.0,1,0.0,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.52,25.9,0,0.0,0.69,24.0,0.0,2,0.0,0,N/A,0.0,33.3,0.0,0,N/A,1,0.0
TRANSITION,0.0,6.7,0,0.0,0.0,8.3,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.33,20.0,0,66.7,2.0,16.7,100.0,2,100.0,0,N/A,0.0,33.3,0.0,0,N/A,1,0.0
Catch & Shoot,0.0,20.0,0,0.0,0.0,25.0,0.0,0,N/A,3,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [78]:
ruiStats = PPP(ovr_player_data['rui'])
ruiStats.to_csv('!PPP Stats/ruiStats.csv')
ruiStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,3.3,0,0.0,0.0,2.0,0.0,1,0.0,0,N/A,0.0,11.1,0.0,1,0.0,0,N/A
PNR Screener,1.5,6.7,0,75.0,1.5,7.8,75.0,4,75.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.1,21.3,0,45.5,1.01,19.4,44.4,9,44.4,0,N/A,1.39,32.0,50.0,0,N/A,2,50.0
TRANSITION,1.2,19.4,1,50.0,1.16,16.9,40.0,5,40.0,0,N/A,1.33,33.3,66.7,3,66.7,0,N/A
Attacking Closeouts,1.09,18.3,0,54.5,1.2,19.6,60.0,10,60.0,0,N/A,0.0,11.1,0.0,0,N/A,1,0.0
Catch & Shoot,0.75,20.0,1,27.3,0.9,19.6,33.3,0,N/A,9,33.3,0.0,22.2,0.0,0,N/A,2,0.0


In [79]:
newRussellStats = PPP(new_player_data['russell'])
newRussellStats.to_csv(f'Russell/PPP Stats/{newGame}.csv')
newRussellStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.75,50.0,1,100.0,2.0,50.0,100.0,2,100.0,0,N/A,1.5,50.0,100.0,0,N/A,1,100.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,12.5,0,0.0,0.0,25.0,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,3.0,12.5,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,3.0,25.0,100.0,0,N/A,1,100.0
Catch & Shoot,0.0,12.5,0,0.0,0.0,25.0,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [80]:
russellStats = PPP(ovr_player_data['russell'])
russellStats.to_csv('!PPP Stats/russellStats.csv')
russellStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.87,41.1,2,44.4,0.76,36.0,36.4,8,50.0,3,0.0,1.01,49.4,57.1,3,100.0,4,25.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,2.5,3.8,0,100.0,2.0,3.0,100.0,1,100.0,0,N/A,3.0,5.0,100.0,0,N/A,1,100.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.91,18.6,1,37.5,0.89,23.9,33.3,5,20.0,1,100.0,1.0,10.0,50.0,1,100.0,1,0.0
TRANSITION,0.86,13.2,2,60.0,0.67,9.1,33.3,1,100.0,2,0.0,1.0,20.0,100.0,2,100.0,0,N/A
Attacking Closeouts,3.0,1.9,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,3.0,5.0,100.0,0,N/A,1,100.0
Catch & Shoot,0.82,20.8,0,27.3,0.33,27.3,11.1,0,N/A,9,11.1,3.0,10.0,100.0,0,N/A,2,100.0


In [81]:
newSchroderStats = PPP(new_player_data['schroder'])
newSchroderStats.to_csv(f'Schroder/PPP Stats/{newGame}.csv')
newSchroderStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.8,27.8,1,50.0,1.0,15.4,50.0,2,50.0,0,N/A,0.67,60.0,50.0,2,50.0,0,N/A
PNR Screener,2.0,5.6,0,100.0,2.0,7.7,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,5.6,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,20.0,0.0,1,0.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,0.0,5.6,0,0.0,0.0,7.7,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,5.6,0,0.0,0.0,7.7,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.69,16.0,0,0.0,0.0,7.7,0.0,1,0.0,0,N/A,1.06,37.6,0.0,1,0.0,0,N/A
Attacking Closeouts,0.0,5.6,0,0.0,0.0,7.7,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.5,33.3,0,50.0,1.5,46.2,50.0,0,N/A,6,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [82]:
schroderStats = PPP(ovr_player_data['schroder'])
schroderStats.to_csv('!PPP Stats/schroderStats.csv')
schroderStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.79,40.0,4,45.0,0.89,31.0,44.4,9,44.4,0,N/A,0.73,48.4,45.5,7,57.1,4,25.0
PNR Screener,1.0,3.3,0,50.0,2.0,3.4,100.0,1,100.0,0,N/A,0.0,3.2,0.0,0,N/A,1,0.0
DHO Ball Handler,0.26,6.5,0,0.0,0.53,6.5,0.0,1,0.0,0,N/A,0.0,6.5,0.0,2,0.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,0.0,1.7,0,0.0,0.0,3.4,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.5,6.7,0,25.0,0.0,6.9,0.0,2,0.0,0,N/A,1.0,6.5,50.0,1,100.0,1,0.0
TRANSITION,1.24,17.5,0,28.6,1.0,6.9,50.0,2,50.0,0,N/A,1.29,27.5,20.0,1,0.0,4,25.0
Attacking Closeouts,0.0,5.0,0,0.0,0.0,3.4,0.0,1,0.0,0,N/A,0.0,6.5,0.0,1,0.0,1,0.0
Catch & Shoot,1.15,21.7,0,38.5,1.25,41.4,41.7,0,N/A,12,41.7,0.0,3.2,0.0,0,N/A,1,0.0


In [83]:
newVandoStats = PPP(new_player_data['vando'])
newVandoStats.to_csv(f'Vando/PPP Stats/{newGame}.csv')
newVandoStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,25.0,0,0.0,0.0,25.0,0.0,1,0.0,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,2.0,25.0,0,100.0,2.0,25.0,100.0,1,100.0,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,25.0,0,0.0,0.0,25.0,0.0,0,N/A,1,0.0,N/A,N/A,N/A,0,N/A,0,N/A


In [84]:
vandoStats = PPP(ovr_player_data['vando'])
vandoStats.to_csv('!PPP Stats/vandoStats.csv')
vandoStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,12.5,0,0.0,0.0,16.7,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,2.27,11.0,0,N/A,2.27,14.7,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,3.0,12.5,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,3.0,50.0,100.0,0,N/A,1,100.0
TRANSITION,2.08,36.0,0,100.0,2.0,33.3,100.0,2,100.0,0,N/A,2.27,44.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,12.5,0,0.0,0.0,16.7,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [85]:
newWalkerStats = PPP(new_player_data['walker'])
newWalkerStats.to_csv(f'Walker/PPP Stats/{newGame}.csv')
newWalkerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,2.0,33.3,0,100.0,2.0,33.3,100.0,1,100.0,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,33.3,1,N/A,0.0,33.3,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,33.3,0,0.0,0.0,33.3,0.0,0,N/A,1,0.0,N/A,N/A,N/A,0,N/A,0,N/A


In [86]:
walkerStats = PPP(ovr_player_data['walker'])
walkerStats.to_csv('!PPP Stats/walkerStats.csv')
walkerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.82,21.2,1,33.3,1.39,15.2,100.0,1,100.0,0,N/A,0.0,50.0,0.0,2,0.0,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,4.3,0,0.0,0.0,5.3,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.67,13.0,0,33.3,0.0,10.5,0.0,2,0.0,0,N/A,2.0,25.0,100.0,1,100.0,0,N/A
Attacking Closeouts,1.55,16.9,1,100.0,1.04,15.2,100.0,1,100.0,0,N/A,3.0,25.0,100.0,0,N/A,1,100.0
Catch & Shoot,1.09,47.8,0,36.4,1.09,57.9,36.4,0,N/A,11,36.4,N/A,0.0,N/A,0,N/A,0,N/A


In [87]:
newTeamStats = PPP(new_player_data['team'])
newTeamStats.to_csv(f'Team/PPP Stats/{newGame}.csv')
newTeamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.13,27.9,3,54.8,1.26,20.7,61.1,16,62.5,2,50.0,1.0,45.2,46.2,10,50.0,3,33.3
PNR Screener,1.0,8.1,1,33.3,1.0,11.4,33.3,6,33.3,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,0.7,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,2.5,0.0,1,0.0,0,N/A
DHO Screener,0.0,0.7,0,0.0,0.0,1.0,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,1.06,1.4,0,0.0,0.0,1.0,0.0,1,0.0,0,N/A,2.27,2.2,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.91,22.6,1,39.3,0.78,20.1,35.3,17,35.3,0,N/A,1.14,28.6,45.5,4,75.0,7,28.6
TRANSITION,1.32,8.4,0,42.9,1.18,8.0,40.0,5,40.0,0,N/A,1.6,9.4,50.0,2,50.0,0,N/A
Attacking Closeouts,1.22,8.4,1,60.0,1.3,8.8,71.4,7,71.4,0,N/A,1.0,7.5,33.3,0,N/A,3,33.3
Catch & Shoot,1.24,12.5,0,41.2,1.24,17.7,41.2,0,N/A,17,41.2,N/A,0.0,N/A,0,N/A,0,N/A


In [88]:
newOppStats = PPP(new_player_data['opp'])
newOppStats.to_csv(f'Opp/PPP Stats/{newGame}.csv')
newOppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.23,13.0,2,57.1,1.25,11.9,62.5,8,62.5,0,N/A,1.21,15.2,50.0,2,50.0,4,50.0
PNR Screener,1.55,2.7,0,66.7,1.0,2.1,50.0,2,50.0,0,N/A,2.13,3.8,100.0,0,N/A,1,100.0
DHO Ball Handler,0.74,7.6,0,30.0,0.67,9.6,33.3,6,50.0,3,0.0,1.06,3.8,0.0,0,N/A,1,0.0
DHO Screener,0.67,6.3,0,33.3,0.0,1.1,0.0,0,N/A,1,0.0,0.75,16.3,37.5,5,60.0,3,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.25,28.0,5,59.4,1.06,28.0,52.4,17,47.1,4,75.0,1.6,28.1,72.7,7,71.4,4,75.0
TRANSITION,0.93,10.5,0,40.0,1.4,5.3,60.0,4,50.0,1,100.0,0.7,20.4,30.0,4,50.0,6,16.7
Attacking Closeouts,1.0,3.5,2,66.7,0.67,3.2,50.0,2,50.0,0,N/A,1.5,4.1,100.0,0,N/A,1,100.0
Catch & Shoot,1.2,15.7,0,40.9,1.2,23.9,40.9,1,100.0,21,38.1,N/A,0.0,N/A,0,N/A,0,N/A


In [89]:
teamStats = PPP(ovr_player_data['team'])
teamStats.to_csv('!PPP Stats/teamStats.csv')
teamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.01,23.0,12,47.3,0.98,16.3,43.6,43,51.2,12,16.7,1.04,36.1,50.9,37,56.8,18,38.9
PNR Screener,1.28,6.9,1,53.3,1.33,9.6,55.6,25,52.0,2,100.0,0.67,1.5,33.3,1,100.0,2,0.0
DHO Ball Handler,1.01,1.7,0,44.4,1.19,1.5,60.0,5,60.0,0,N/A,0.75,2.0,25.0,3,0.0,1,100.0
DHO Screener,0.71,1.7,1,37.5,0.0,0.8,0.0,2,0.0,0,N/A,1.02,3.5,50.0,6,50.0,0,N/A
DBL Ball Handler,1.06,0.3,0,0.0,0.0,0.3,0.0,1,0.0,0,N/A,2.27,0.4,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.04,21.5,8,45.3,0.95,20.4,40.6,56,39.3,8,50.0,1.19,23.5,52.4,18,72.2,24,37.5
TRANSITION,1.29,16.3,7,54.0,1.31,13.9,52.8,30,60.0,6,16.7,1.26,20.9,55.6,15,73.3,12,33.3
Attacking Closeouts,1.13,6.7,1,51.4,1.13,7.9,53.8,25,52.0,1,100.0,1.11,4.6,44.4,4,50.0,5,40.0
Catch & Shoot,1.12,13.4,1,37.7,1.11,19.1,37.5,0,N/A,72,37.5,1.2,2.5,40.0,0,N/A,5,40.0


In [90]:
oppStats = PPP(ovr_player_data['opp'])
oppStats.to_csv('!PPP Stats/oppStats.csv')
oppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.28,12.2,5,53.1,1.38,12.8,56.8,28,57.1,16,56.2,1.07,11.3,45.0,8,37.5,12,50.0
PNR Screener,0.94,1.9,1,44.4,0.86,1.8,42.9,6,50.0,1,0.0,1.05,2.1,50.0,0,N/A,2,50.0
DHO Ball Handler,0.91,7.0,1,35.0,0.9,7.7,36.7,16,37.5,14,35.7,0.94,5.7,30.0,4,25.0,6,33.3
DHO Screener,0.86,5.8,1,35.3,0.68,1.5,20.0,4,25.0,1,0.0,0.9,13.4,37.9,15,40.0,14,35.7
DBL Ball Handler,2.06,0.6,0,66.7,2.27,0.2,N/A,0,N/A,0,N/A,2.0,1.3,66.7,1,0.0,2,100.0
DBL Screener,1.5,0.3,0,50.0,1.5,0.5,50.0,1,0.0,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.08,23.4,15,51.3,1.09,20.0,50.8,57,49.1,8,62.5,1.07,29.2,51.9,26,69.2,26,34.6
TRANSITION,1.29,19.3,6,57.3,1.43,14.5,63.3,36,66.7,13,53.8,1.15,27.5,51.9,25,68.0,29,37.9
Attacking Closeouts,1.12,4.5,5,52.4,0.88,5.8,41.2,12,41.7,5,40.0,2.2,2.2,100.0,1,100.0,3,100.0
Catch & Shoot,1.18,12.6,0,40.3,1.19,19.6,40.8,3,100.0,73,38.4,0.0,0.4,0.0,0,N/A,1,0.0
